In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy
import scipy
import matplotlib
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import pytz
import multiprocessing as mpp
import pickle
import os
import time
import numba
import subprocess
#
#import pyspark
import h5py
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib

#
#data_file_name = 'data/mazama_usage_20200506_tool8.out'
#data_file_name = 'data/sacct_sherlock_out_serc2020_05_08.out'
data_file_name = 'data/serc_usage_20200914.out'
#
pkl_name = "{}.pkl".format(os.path.splitext(data_file_name)[0])
h5_name = "{}.h5".format(os.path.splitext(data_file_name)[0])
#

In [3]:
#sacct_FORMAT = 'User,Group,GID,Jobname,JobID,JobIDRaw,partition,state,time,ncpus,nnodes,Submit,Eligible,start,end,elapsed,SystemCPU,UserCPU,TotalCPU,NTasks,CPUTimeRaw,Suspended,ReqGRES,AllocGRES,ReqTRES,AllocTRES'
#format_list = [str(s) for s in sacct_FORMAT.split(',')]
#print('** ', format_list)
#
def date_to_string(dt):
    return "{}-{}-{}T{}:{}:{}".format(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
#
format_list = ['User', 'Group', 'GID', 'Jobname', 'JobID', 'JobIDRaw', 'partition', 'state', 'time', 'ncpus',
               'nnodes', 'Submit', 'Eligible', 'start', 'end', 'elapsed', 'SystemCPU', 'UserCPU',
               'TotalCPU', 'NTasks', 'CPUTimeRaw', 'Suspended', 'ReqGRES', 'AllocGRES', 'ReqTRES',
               'AllocTRES']
#
start_time = dtm.date(2020,4,15)
end_time   = dtm.date(2020,11,15)
end_time   = dtm.date(2020,7,15)
#
# construct start,end times array:
start_end_times = [(start_time, min(start_time + dtm.timedelta(days=30), end_time))]
options  = [('group', 'oneillm'), ('delimiter', '"|"'), ('allusers',None)]
#
while start_end_times[-1][1] < end_time:
    start_end_times += [[start_end_times[-1][1], min(start_end_times[-1][1] + dtm.timedelta(days=30), end_time) ]]
#
# NOTE: Sherlock will only allow one month of sacct query, so we break this up. I usually just do 15th - 15th,
#. but maybe we can be a bit more sophisticated here.
#  TODO: include time, but I'm not sure of the correct format right now (ie, yyyy-mm-ddThh:mm:ss, yyyy-mm-dd hh:mm:ss,
#. and then )
print('** Start_end_times: ', start_end_times)
#

# TODO: these are cut-pasted from .sh; reformat.
sacct_format = ','.join(format_list)
#

#
options_str = ''

for (op,vl) in options:
    if vl is None:
        options_str += '--{}'.format(op)
    else:
        options_str += ' --{}={} '.format(op,vl)
#
print('** options: ', options_str)

** Start_end_times:  [(datetime.date(2020, 4, 15), datetime.date(2020, 5, 15)), [datetime.date(2020, 5, 15), datetime.date(2020, 6, 14)], [datetime.date(2020, 6, 14), datetime.date(2020, 7, 14)], [datetime.date(2020, 7, 14), datetime.date(2020, 7, 15)]]
** options:   --group=oneillm  --delimiter="|" --allusers


In [4]:
#
sacct_out = ''
Ss = []
for k, (start, stop) in enumerate(start_end_times):
    sacct_str = 'srun sacct {} {} -p --starttime={} --endtime={} --format={} '.format( ('--noheader' if k>0 else ''), 
                                    options_str, start, stop, sacct_format )
    #
    #sacct_str_list = sacct_str.split()
    #break
    print('** [{}]: {}\n'.format(k, sacct_str))
    #
    S = subprocess.run(sacct_str.split(), stdout=subprocess.PIPE)
    # TODO: parallelize...
    Ss += S.stdout.decode()
    sacct_out += S.stdout.decode()



** [0]: srun sacct   --group=oneillm  --delimiter="|" --allusers -p --starttime=2020-04-15 --endtime=2020-05-15 --format=User,Group,GID,Jobname,JobID,JobIDRaw,partition,state,time,ncpus,nnodes,Submit,Eligible,start,end,elapsed,SystemCPU,UserCPU,TotalCPU,NTasks,CPUTimeRaw,Suspended,ReqGRES,AllocGRES,ReqTRES,AllocTRES 

** [1]: srun sacct --noheader  --group=oneillm  --delimiter="|" --allusers -p --starttime=2020-05-15 --endtime=2020-06-14 --format=User,Group,GID,Jobname,JobID,JobIDRaw,partition,state,time,ncpus,nnodes,Submit,Eligible,start,end,elapsed,SystemCPU,UserCPU,TotalCPU,NTasks,CPUTimeRaw,Suspended,ReqGRES,AllocGRES,ReqTRES,AllocTRES 

** [2]: srun sacct --noheader  --group=oneillm  --delimiter="|" --allusers -p --starttime=2020-06-14 --endtime=2020-07-14 --format=User,Group,GID,Jobname,JobID,JobIDRaw,partition,state,time,ncpus,nnodes,Submit,Eligible,start,end,elapsed,SystemCPU,UserCPU,TotalCPU,NTasks,CPUTimeRaw,Suspended,ReqGRES,AllocGRES,ReqTRES,AllocTRES 

** [3]: srun sacct -

In [12]:
print('** ', len(sacct_out.split('\n')))

**  10916


In [13]:
for rw in sacct_out.split('\n')[0:10]:
    print('** ', rw)

**  User"|"Group"|"GID"|"JobName"|"JobID"|"JobIDRaw"|"Partition"|"State"|"Timelimit"|"NCPUS"|"NNodes"|"Submit"|"Eligible"|"Start"|"End"|"Elapsed"|"SystemCPU"|"UserCPU"|"TotalCPU"|"NTasks"|"CPUTimeRAW"|"Suspended"|"ReqGRES"|"AllocGRES"|"ReqTRES"|"AllocTRES"|"
**  haofu"|"oneillm"|"328022"|"SWE"|"65485294_20"|"65485335"|"oneillm"|"COMPLETED"|"3-00:00:00"|"1"|"1"|"2020-04-14T20:41:14"|"2020-04-14T20:41:14"|"2020-04-14T20:42:10"|"2020-04-17T01:56:02"|"2-05:13:52"|"04:54:12"|"1-23:40:38"|"2-04:34:50"|""|"191632"|"00:00:00"|""|""|"billing=1,cpu=1,mem=1G,node=1"|"billing=1,cpu=1,energy=41149423,mem=1G,node=1"|"
**  "|""|""|"batch"|"65485294_20.batch"|"65485335.batch"|""|"COMPLETED"|""|"1"|"1"|"2020-04-14T20:42:10"|"2020-04-14T20:42:10"|"2020-04-14T20:42:10"|"2020-04-17T01:56:02"|"2-05:13:52"|"04:54:12"|"1-23:40:38"|"2-04:34:50"|"1"|"191632"|"00:00:00"|""|""|""|"cpu=1,mem=1G,node=1"|"
**  "|""|""|"extern"|"65485294_20.extern"|"65485335.extern"|""|"COMPLETED"|""|"1"|"1"|"2020-04-14T20:42:10"|"2

In [7]:
print('** ', sacct_str_list)

**  ['srun', 'sacct', '--group=oneillm', '--delimiter="|"', '--allusers', '-p', '--starttime=2020-04-15', '--endtime=2020-05-15', '--format=User,Group,GID,Jobname,JobID,JobIDRaw,partition,state,time,ncpus,nnodes,Submit,Eligible,start,end,elapsed,SystemCPU,UserCPU,TotalCPU,NTasks,CPUTimeRaw,Suspended,ReqGRES,AllocGRES,ReqTRES,AllocTRES']


In [8]:
XX = subprocess.run(sacct_str_list, stdout=subprocess.PIPE)

In [10]:
XX.stdout.decode()[0:10]

'User"|"Gro'

In [ ]:
xx = hpc_lib.SACCT_data_direct(group='oneillm', start_date=dtm.datetime(2020,8,1), end_date=dtm.datetime(2020,10,31))

/scratch/users/myoder96/HPC_analytics/hpc_lib.py:517: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "process_row" failed type inference due to: non-precise type pyobject
During: typing of argument at /scratch/users/myoder96/HPC_analytics/hpc_lib.py (523)

File "hpc_lib.py", line 523:
    def process_row(self, rw, headers=None, RH=None):
        <source elided>
        #
        headers = (headers or self.headers)
        ^

  @numba.jit
/scratch/users/myoder96/HPC_analytics/hpc_lib.py:517: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "process_row" failed type inference due to: non-precise type pyobject
During: typing of argument at /scratch/users/myoder96/HPC_analytics/hpc_lib.py (523)

File "hpc_lib.py", line 523:
    def process_row(self, rw, headers=None, RH=None):
        <source elided>
        #
        headers = (headers or self.headers)
        ^

  @numba.jit
/scratch/u

In [ ]:
fg = plt.figure(figsize=(12,10))
ax1 = plt.subplot('211')
ax2 = plt.subplot('212')

#
zz = xx.compute_vol_distributions_report(ax1=ax1, ax2=ax2)
ax1.grid()
ax2.grid()